<a href="https://colab.research.google.com/github/BandaVallary/Movie_Recommendation_System/blob/kelvin/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation System

!["Image of Donald Draper in a Movie Theater"](https://github.com/BandaVallary/Movie_Recommendation_System/blob/kelvin/images/dondraperinmovietheater.jpg?raw=1)

## Overview

This project aims to develop a recommendation system that provides personalized movie recommendations based on user ratings. Utilizing the [MovieLens dataset](https://web.archive.org/web/20240828133414/https://grouplens.org/datasets/movielens/latest/) from the GroupLens research lab at the University of Minnesota, the model will be trained on a subset of the dataset containing 100,000 user ratings. The primary objective is to build a machine learning model that can accurately recommend the top 5 movies to a user, based on their ratings of other movies. This system can be valuable for streaming platforms and movie enthusiasts, offering tailored movie suggestions to enhance user experience and engagement.

The project will involve several steps, including data cleaning, exploratory data analysis, feature engineering, model selection, and evaluation.

Throughout this project, we will also explore the relationships between different variables and their impact on movie recommendations. This will help us gain insights into user preferences and identify potential areas for improvement. Overall, this project has the potential to provide valuable insights and practical applications for the entertainment industry. By developing a recommendation system that can accurately suggest movies, streaming platforms can better engage their users, improve customer satisfaction, and increase viewership.

## Business Understanding

The entertainment industry, particularly streaming platforms, is highly competitive, with companies constantly striving to enhance user engagement and satisfaction. One of the major challenges faced by these platforms is providing personalized content recommendations that keep users engaged and reduce churn rates.

According to recent studies, personalized recommendations can significantly increase user engagement and satisfaction, leading to higher retention rates and increased viewership. This highlights the need for a robust recommendation system that can accurately suggest movies based on user preferences. By building a recommendation system that can provide top 5 movie recommendations to users based on their ratings of other movies, streaming platforms can offer a more tailored viewing experience.

The business value of this project lies in its ability to help streaming platforms improve their content recommendation strategies, increase user satisfaction, and reduce churn rates. By developing a recommendation system that can accurately suggest movies, platforms can better engage their users, leading to increased viewership and subscription renewals. This can provide a competitive edge in the highly competitive entertainment industry, ultimately driving revenue growth and customer loyalty.

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357289 sha256=d25bc5a5695be34387c2c6606387852d13de64ba12a31f146892ee5ab3db53ee
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')

from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset

## Reading the Data

In [4]:
# Loading the data
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')
tags = pd.read_csv('/content/tags.csv')

## Tidying the data (ratings df)

1. Check data types and figure out which figures are numerical and which are categorical.
2. Check for null values.
3. Check for duplicate values

I check for null values in the dataset. There are none.

In [5]:
# checking for null values
ratings.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0


I then check for duplicates. There are none.

In [6]:
# check for duplicates
ratings.duplicated().sum()

0

In [7]:
# check data types
ratings.dtypes

,0
userId,int64
movieId,int64
rating,float64
timestamp,int64


In [8]:
# Drop the timestamp column
ratings.drop('timestamp', axis=1, inplace=True)

Now, I'll add the `title` column from the `movies` dataframe to the `ratings` dataframe based on the common `movieId` column.

In [9]:
# extracting the title from the `movies` df based on the `movieId`
ratings_with_titles = ratings.merge(movies[['movieId', 'title']], on='movieId', how='left')

I'm starting with the `ratings_with_titles` df for my basic recommendation model because it contains key user-item interactions, which are essential for collaborative filtering. This dataframe shows which users rated which movies and their ratings, allowing me to identify patterns in preferences.

I'll use collaborative filtering, focusing on either:

- User-Based: Recommending movies based on the preferences of similar users.
- Item-Based: Suggesting movies based on similarities between items that users have liked.

Using `ratings_with_titles` gives me a solid foundation for identifying these relationships, and I can later integrate `movies` and `tags` dataframes for more nuanced recommendations.

Now, I'll create a scikit surprise dataset from the `ratings` dataframe.

In [10]:
# reading the values as scikit surprise dataset
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [11]:
# parameter grid for SVD
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}

In [12]:
# create the grid search object
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3, joblib_verbose=5, n_jobs=-1)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.1min finished


In [13]:
# Get the best parameters
print(gs.best_score)
print(gs.best_params)

{'rmse': 0.8822700492656498, 'mae': 0.6815579020395642}
{'rmse': {'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}, 'mae': {'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}}


In [14]:
# Cross validation with KNNBasic
# Parameter grid for KNNBasic
param_grid = {
    'k': [20, 40, 60],
    'min_k': [1, 2, 3],
    'sim_options': {
        'name': ['cosine', 'msd', 'pearson'],
        'user_based': [True, False]
    }
}

# Create the grid search object
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=3, n_jobs=-1)

# Fit the data
gs.fit(data)

# Get the best parameters and scores
print("Best RMSE:", gs.best_score['rmse'])
print("Best MAE:", gs.best_score['mae'])
print("Best Parameters:", gs.best_params['rmse'])

Best RMSE: 0.911176402700666
Best MAE: 0.7008877343061753
Best Parameters: {'k': 60, 'min_k': 3, 'sim_options': {'name': 'msd', 'user_based': False}}
